**BRONZE QC/QA DASHBOARD NOTES**


![image.png](attachment:538a4afc-2bcc-4798-90ce-405ea8c30a5d.png)

Go to `Edit Connector -> Custom Query` and insert the following code

In [ ]:
WITH RawData AS (
  SELECT
    *,
    CASE
      WHEN @Variable = 'LWIN_1_1_1' THEN CAST(LWIN_1_1_1 AS FLOAT64)
      WHEN @Variable = 'SHF_1_1_1' THEN CAST(SHF_1_1_1 AS FLOAT64)
      WHEN @Variable = 'SWC_1_1_1' THEN CAST(SWC_1_1_1 AS FLOAT64)
      WHEN @Variable = 'SWIN_1_1_1' THEN CAST(SWIN_1_1_1 AS FLOAT64)
      WHEN @Variable = 'CE_1_1_1' THEN CAST(CE_1_1_1 AS FLOAT64)
# @Variable list all variables available as defined in your Parameter
      ELSE NULL
    END AS dynamic_value  # Here it creates the calculated field
  FROM `manglaria_lakehouse_ds.carbon_flux_eddycov_meteo` # Look in your table
),
quartiles AS (
  SELECT
    *,
# Calculates quartiles used in boxplots, % Null data, and % Outliers
    PERCENTILE_CONT(dynamic_value, 0.25) OVER() as Q1,
    PERCENTILE_CONT(dynamic_value, 0.50) OVER() as Q2,
    PERCENTILE_CONT(dynamic_value, 0.75) OVER() as Q3
  FROM RawData
)
  SELECT
    *,
    (Q3 - Q1) AS IQR,
    CASE
      WHEN dynamic_value > (Q3 + 1.5 * (Q3 - Q1)) THEN 1
      WHEN dynamic_value < (Q1 - 1.5 * (Q3 - Q1)) THEN 1
      ELSE 0
    END AS is_outlier, # Define an outlier flag for each feature
    CASE
      WHEN dynamic_value IS NULL THEN 1
      ELSE 0
    END AS is_null, # Define a null flag for each feature
    CASE
    WHEN dynamic_value > (AVG(dynamic_value) OVER() + 3 * STDDEV_POP(dynamic_value) OVER())
      OR dynamic_value < (AVG(dynamic_value) OVER() - 3 * STDDEV_POP(dynamic_value) OVER())
    THEN 1
    ELSE 0
    END AS is_extreme # Define an extreme flag for each feature
FROM quartiles


The following are necessary calculated fields and their formulas for the SCORECARDS

a) Extreme values

    SUM(is_extreme)/COUNT(dynamic_value)

b) Null data

    SUM(is_null)/COUNT(is_null)

c) Outliers

    SUM(is_outlier)/COUNT(dynamic_value)


The following are necessary calculated fields and their formulas for the BOXPLOT

d) Maximum

    MAX(dynamic_value)

e) Minimum

    MIN(dynamic_value)

f) Quartile 25

    PERCENTILE(dynamic_value,0.25)

g) Quartile 50

    PERCENTILE(dynamic_value,0.50)

h) Quartile 75

    PERCENTILE(dynamic_value,0.75)

Observations:

Line chart X-axis shows `timestamp (Date hour)` and Y-axis shows `AVG(dynamic value)`. There is one record every 30 minutes in this case.
